In [1]:
%load_ext autoreload
%autoreload 2

from netty.netty import Netty
import img_utils as im
import numpy as np
from netty import netty_utils as nutil
from IPython.display import clear_output

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
net = Netty()
net.clear()
net.args["style_layers"] = [1,4,7]
net.build()

/usr/local/lib/python3.5/dist-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
a = im.load("data/tex/flame_braini075.jpg")
b = im.load("data/tex/exp_2.jpg")
b2 = im.load("data/tex/exp_1.jpg")
c = im.load("data/tex/br_5.jpg")
d = im.histmatch(255-c,b)
e = im.histmatch(c,b)
f = im.load("data/tex/flame_brain.jpg")


fsize = [4134+100,2480+100]

xs = [
    im.load("data/x/bok_1.jpg"),
    im.load("data/x/bok_flame_braini075.jpg"),
    im.load("data/x/bok_4.jpg"),
    im.load("data/x/bok_2.jpg"),
    im.load("data/x/flame075.jpg")
]

imgs_ = [
    [a,b],
    [a],
    [d,b2],
    [d,b],
    [f]
]
masks = None
scales_ = [
    [im.propscale(fsize,a.shape[:2]), im.propscale(fsize,b.shape[:2])],
    [im.propscale(fsize,a.shape[:2])],
    [im.propscale(fsize,d.shape[:2])*0.75, im.propscale(fsize,b.shape[:2])],
    [im.propscale(fsize,d.shape[:2])*0.75, im.propscale(fsize,b.shape[:2])],
    [im.propscale(fsize,f.shape[:2])*0.75]
         ]
w = None

In [16]:
g = im.load("data/tex/b5_05-e2_1.jpg")
h = im.load("data/tex/b5_05i_h300-e2_1.jpg")

fsize = [4134+100, 2480+100]

imgs2_ = [
    [h],
    [g]
]
masks = None
scales2_ = [
    [im.propscale(fsize,h.shape[:2])],
    [im.propscale(fsize,g.shape[:2])]
         ]
w = None

In [23]:
x2s = [prefn(i_,s_) for i_, s_ in zip(imgs2_,scales2_)]

fn(x2s[0],imgs2_[0],scales2_[0])
fn(x2s[1],imgs2_[1],scales2_[1])


fn(xs[0],imgs_[0],scales_[0])
fn(xs[1],imgs_[1],scales_[1])
fn(xs[2],imgs_[2],scales_[2])
fn(xs[3],imgs_[3],scales_[3])
fn(xs[4],imgs_[4],scales_[4])



IndexError: list index out of range

In [22]:
def fn(x,imgs,scales):
    render_iters = 20
    render_maxfun = 20

    hh = 2480+100
    ww = 4134+100
    w2 = ww // 2
    ovr = 600
    ov2 = ovr // 2

    x = im.size(x,(ww,hh))
    x = np.array(x)

    net.args["size"] = [w2+ov2,hh]
    net.args["maxfun"] = render_maxfun
    patches = [[[0,hh],[0,w2+ov2]],[[0,hh],[w2-ov2,ww]]]

    for j in range(render_iters):
        for i,p in enumerate(patches):
            print(p[0][0],p[0][1],p[1][0],p[1][1])

            net.set_style(imgs,masks,scales,w)
            net.set_x0(x[p[0][0]:p[0][1],p[1][0]:p[1][1]])
            print("Setup...")
            net.setup()
            print("Render!")
            x1=net.render()
            clear_output()
        #     
            if i == 0:
                x[p[0][0]:p[0][1],p[1][0]:p[1][1]-ovr] = x1[:,:-ovr]
                for y in range(ovr):
                    x[p[0][0]:p[0][1],p[1][1]-ovr+y] = x[p[0][0]:p[0][1],p[1][1]-ovr+y]*((y+1)/ovr) + x1[:,-ovr+y]*((ovr-y-1)/ovr)
            else:
                x[p[0][0]:p[0][1],p[1][0]+ovr:p[1][1]] = x1[:,ovr:]
                for y in range(ovr):
                    x[p[0][0]:p[0][1],p[1][0]+y] = x[p[0][0]:p[0][1],p[1][0]+y]*((y+1)/ovr) + x1[:,y]*((ovr-y-1)/ovr)

            im.save_frame(x,"data/bin",q=95)

In [20]:
def prefn(imgs,scales):
    x = None
    s,m = nutil.incremental([3120,1902],512//2,1.2,300)
    for s_,m_ in zip(s,m):
        print(s_)
        print(m_)
        net.args["size"] = s_
        net.args["maxfun"] = m_
        print("Setting style...")
        net.set_style(imgs,masks,scales,w)
        net.set_x0(x)
        print("Setup...")
        net.setup()


        print("Render!")
        x=net.render()
        clear_output()
    im.save_frame(x,"data/bin",q=95)
    return x

In [4]:
x = np.array(x)
x[p[0][0]:p[0][1],p[1][0]:p[1][1]-ovr] = x1[:,:-ovr]

NameError: name 'x' is not defined

In [5]:
print(patches[0][0][0],patches[0][0][1],patches[0][1][0],patches[0][1][1]-ovr)
print(x[patches[0][0][0]:patches[0][0][1],patches[0][1][0]:patches[0][1][1]-ovr].shape)
print(x1[:,:-ovr].shape)
print(x1.shape)
print(x.shape)


net.args["size"]

NameError: name 'patches' is not defined

In [8]:
im.show(x)

NameError: name 'x' is not defined